# Here you need to add the paper title, and then a small block of general explanation for a binder

# Experiments on neuromuscular efficiency of respiratory muscles

Import libraries

In [1]:
from scipy import interpolate as interp
import scipy
import pandas as pd
import glob
import os
import sys
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from datetime import datetime
%matplotlib widget

# The cell below will need to be changed for a generalized binder

In [ ]:
# import helper functions
import resurfemg.preprocessing.ecg_removal as ecg_rm
import resurfemg.preprocessing.envelope as evl
import resurfemg.preprocessing.filtering as filt
import resurfemg.postprocessing.features as feat

from resurfemg.data_connector.config import Config
config = Config()

In [ ]:
# get new changes in our library i.e. the tmsisdk
from resurfemg.data_connector.tmsisdk_lite import Poly5Reader

# Initiation of output folder for experiments

In [ ]:
# Output data - General path to dir for saving .csvs and plots
main_output_dir = config.get_directory('preprocessed') + \
                    '/output_dir'
if not os.path.exists(main_output_dir):
    os.makedirs(main_output_dir)

The below cell allows you to pick whether to display and interact with the data in seconds or samples.

In [ ]:
y_axis = widgets.Dropdown(
    options=["Samples", "Seconds"],
    value='Seconds',
    description="Select View Option",
    disabled=False,
)
display(y_axis)

The below cell confirms your choice.

In [ ]:
time_view = y_axis.value
time_view

<a id='#patient_select'>Input patient file path</a>

In [ ]:
# Root directory for patient data
root_patient_data_directory = \
    config.get_directory('root_patient_data_directory')

patient_idx = 0
measurements = []

# Patient data selection

In [ ]:
# Reruns should be done from this cell as the start

<a id='section_patient_step'>Next patient</a>

In [ ]:
patient_folders = \
    glob.glob(root_patient_data_directory + '/**/', recursive=False)
patients = []
prev_measurements = measurements

for folder in patient_folders:
    name = folder.split("\\")[-2]
    patients.append(name)

btn_pt = widgets.Dropdown(  
    options=patients,
    value=patients[patient_idx],
    description='Select patient:',
    disabled=False,
)

date_idx = 0

display(btn_pt)

<a id='section_date_step'>Next date</a>

In [ ]:
patient = btn_pt.value
patient_idx =btn_pt.index

measurement_folders = glob.glob(root_patient_data_directory 
                                + '/' + patient 
                                + '/**/',
                                recursive=False
                                )
measurements = []

for folder in measurement_folders:
    name = folder.split("\\")[-2]
    measurements.append(name)

btn_measurement = widgets.Dropdown(
    options=measurements,
    value=measurements[date_idx],
    description='Select measurement date:',
    disabled=False,
)
display(btn_measurement)

In [ ]:
gate_width = 0.1
gate_twice = False

if (date_idx == 0) & ((patient_idx == 0) | len(prev_measurements) == 0):
    pass
elif (date_idx == 0) & (patient_idx > 0):
    if not os.path.exists(main_output_dir + '/' + patients[patient_idx-1] 
                          + '/' + prev_measurements[-1]  + '/' 
                          + prev_measurements[-1] + '_' 
                          + patients[patient_idx-1] 
                          +'_NME_output_gate_'
                          + str(int(gate_width*1000))+'.csv'):
        print(patients[patient_idx-1], prev_measurements[-1])
        print('Warning: Data of previous patient not stored! '
              +'Do not progress to next patient!')
    else:
        print(patients[patient_idx-1], prev_measurements[-1])
        print('Notification: Data stored appropriately! '
              +'You can progress to the selected date of the new patient!')
else:    
    if not os.path.exists(main_output_dir + '/' + patient + '/' 
                          + measurements[date_idx-1]  + '/' 
                          + measurements[date_idx-1] + '_' + patient 
                          + '_NME_output_gate_'
                          + str(int(gate_width*1000))+'.csv'):
        print(measurements[date_idx-1])
        print('Warning: Data of previous date not stored! '
              +'Do not progress to next date!')
    else:
        print(measurements[date_idx-1])
        print('Notification: Data of previous date stored appropriately! '
              +'You can progress to selected date!')
        


In [ ]:
# Rerun for each new measurement DAY (empty output parameter list)!
measurement = btn_measurement.value
date_idx = btn_measurement.index

# Input data
windows_str = root_patient_data_directory + '/' + patient + '/' + measurement
root_emg_directory = os.path.join(windows_str)

# do not rerun this cell
big_data_list = []
ecg_data_list = []


measurement_idx = -4

<a id='section_peep_step'>Next PEEP step</a>

Now you can pick a file from the list, which have been numbered.

In [ ]:
# Select measurement number
emg_pattern = os.path.join(root_emg_directory, '**/*.Poly5')
emg_and_vent_files = glob.glob(emg_pattern, recursive=True)

emg_files = []
vent_files = []

for file in emg_and_vent_files:
    if 'Draeger' in file:
        vent_files.append(file)
    else:
        emg_files.append(file)
list_of_numbers_strung = []

for i in range(len(emg_files)):
    # list_of_numbers_strung.append(str(i))

    list_of_numbers_strung.append(emg_files[i].split('\\')[-2])


btn = widgets.Dropdown(
    options=list_of_numbers_strung,
    value=str(emg_files[measurement_idx].split('\\')[-2]),
    description='Picked File:',
    disabled=False,
)
display(btn)

In [ ]:
number_chosen = int(btn.index)
measurement_idx = int(btn.index)
emg_file_chosen = emg_files[number_chosen]
vent_file_chosen = vent_files[number_chosen]
print("The files you chose are:\n", emg_file_chosen, '\n', vent_file_chosen)

In [ ]:
# get the poly5 reading
data_emg = Poly5Reader(emg_file_chosen)
data_vent = Poly5Reader(vent_file_chosen)
data_emg_samples = data_emg.samples[:data_emg.num_samples]
emg_fs = data_emg.sample_rate
data_vent_samples = data_vent.samples[:data_vent.num_samples]
vent_fs = data_vent.sample_rate

# Output data path - Patient specific
patient = emg_files[number_chosen].split('\\')[-3].split('/')[-2]
measurement = emg_files[number_chosen].split('\\')[-2]

if not os.path.exists(main_output_dir + '/' + patient + '/'):
    os.makedirs(main_output_dir + '/' + patient + '/')

if not os.path.exists(main_output_dir + '/' + patient 
                      + '/' + measurements[date_idx] + '/'
):
    os.makedirs(main_output_dir + '/' + patient + '/' 
                + measurements[date_idx] + '/')


In [ ]:
y_emg = data_emg_samples
y_vent = data_vent_samples

x_emg = [i/emg_fs for i in range(len(y_emg[0, :]))]
x_emg = [i/vent_fs for i in range(len(y_vent[0, :]))]

In [ ]:
# Load settings from prior run
readme_file_name_prev = config.get_directory('preprocessed') \
                        + '/2023_07_07_Set_Quality_criteria/' \
                        + patient + '/' + measurements[date_idx] + '/' \
                        + measurements[date_idx] + '_' + patient + '_' \
                        + measurement + '_readme.txt'

if not os.path.exists(readme_file_name_prev):
    readme_file_name_prev = config.get_directory('preprocessed') \
                            + '/2023_04_SNR_good/' + patient + '/' \
                            + measurements[date_idx] + '/' \
                            + measurements[date_idx] + '_' + patient + '_' \
                            + measurement + '_readme.txt'
try:
    with open(readme_file_name_prev, 'r') as f:
        readme_str = f.read()
        
        settings_lines = readme_str.split('\n')
        
        
        t_start_prev = float(settings_lines[1].split(' ')[-2])
        t_end_prev = float(settings_lines[2].split(' ')[-2])

        gate_width_prev = float(settings_lines[3].split(' ')[-2])/1000
        gate_threshold_prev = float(settings_lines[4].split(' ')[-1])
        gate_ECG_shift_prev = int(settings_lines[5].split(' ')[-2])
        
        time_shift_prev = float(settings_lines[6].split(' ')[-2])

        Pocc_prominence_factor_prev = float(settings_lines[7].split(' ')[-1])
        EMG_di_prominence_factor_prev = float(settings_lines[8].split(' ')[-1])
        EMG_para_prominence_factor_prev = float(
            settings_lines[9].split(' ')[-1])

        if len(settings_lines) > 12:
            N_gated = int(settings_lines[11].split(' ')[-1])
            if N_gated == 2:
                gate_twice = True
            

        print(readme_str + '\n')
    f.close()
    prev_settings_loaded = True
except FileNotFoundError:
    print("Log file not found:")
    print(readme_file_name_prev)
    prev_settings_loaded = False
except ValueError:
    print("Error while reading log file!")
    prev_settings_loaded = False

if not prev_settings_loaded:
    t_start_prev = x_emg[-1]-100
    t_end_prev = x_emg[-1]-10

    gate_width_prev = 0.10
    gate_threshold_prev = 0.30
    gate_ECG_shift_prev = -10

    time_shift_prev = 0.5                   #Draeger delay
    Pocc_prominence_factor_prev = 0.8
    EMG_di_prominence_factor_prev = 0.01
    EMG_para_prominence_factor_prev = 0.01


In [ ]:
# If necessary: Plot data
if not prev_settings_loaded:
    fig, axis = plt.subplots(nrows=3, ncols=4, figsize=(10, 6))
    axis[0, 0].grid(True)
    axis[0, 0].plot(y_emg[0])
    axis[0, 0].set(title='EMG leads: samples')
    axis[1, 0].plot(y_emg[1])
    axis[2, 0].plot(y_emg[2])

    axis[0, 1].set(title='Draeger: samples')
    axis[0, 1].grid(True)
    axis[0, 1].plot(y_vent[0])
    axis[1, 1].plot(y_vent[1])
    axis[2, 1].plot(y_vent[2])

    axis[0, 2].grid(True)
    axis[0, 2].plot(x_emg, y_emg[0])
    axis[0, 2].set(title='EMG leads: seconds')
    axis[1, 2].plot(x_emg, y_emg[1])
    axis[2, 2].plot(x_emg, y_emg[2])

    axis[0, 3].set(title='Draeger :seconds')
    axis[0, 3].grid(True)
    axis[0, 3].plot(x_emg, y_vent[0])
    axis[1, 3].plot(x_emg, y_vent[1])
    axis[2, 3].plot(x_emg, y_vent[2])

Here pick the part of the signal you want to look at. It is advised to never start at zero, or end after the signal, and the widgets have been constrained to do this.

In [ ]:
start_widget = widgets.BoundedFloatText(
    value=np.floor(t_start_prev),
    min=1,
    max=x_emg[-1],
    step=1,
    description='Start:',
    disabled=False
)
end_widget = widgets.BoundedFloatText(
    value=np.floor(t_end_prev),
    min=1,
    max=x_emg[-1],
    step=1,
    description='End:',
    disabled=False
)
widgets.VBox([start_widget, end_widget])



In [ ]:
start = start_widget.value  
end = end_widget.value  
if start >= end:    
    print('Please choose logical values. You made a mistake.')

In [ ]:
# now examine at the selected window to contain all occlusions

if time_view == 'Seconds':
    start_idx = int(float(start)* emg_fs)
    end_idx = min([int(float(end)*emg_fs), len(y_emg[0,:])-1])
    start_vent_s = int(float(start)* vent_fs)
    end_vent_s = min(
        [int(float(end)* vent_fs), len(y_vent[0,:])-1]
    )

else:
    start_idx = int(float(start))
    end_idx = min([int(float(end)), len(data_emg_samples[0])-1])
    start_vent_s = int(float(start)* vent_fs/emg_fs)
    end_vent_s = min(
        [int(float(start)* vent_fs/emg_fs), 
         len(data_vent_samples[0])-1]
    )
    x_emg = [range(len(data_emg_samples[0]))]
    x_emg = [range(len(data_vent_samples[0]))]


fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(12, 6))
axis[0, 0].set(title='leads in EMG')
axis[0, 0].grid(True)
axis[0, 0].plot(x_emg[int(start_idx):int(end_idx)], 
                y_emg[0][int(start_idx):int(end_idx)])
axis[1, 0].plot(x_emg[int(start_idx):int(end_idx)], 
                y_emg[1][int(start_idx):int(end_idx)])
axis[2, 0].plot(x_emg[int(start_idx):int(end_idx)], 
                y_emg[2][int(start_idx):int(end_idx)])

axis[0, 1].set(title='leads in Draeger')
axis[0, 1].grid(True)
axis[0, 1].plot(
    x_emg[int(start_vent_s):int(end_vent_s)], 
    y_vent[0][int(start_vent_s):int(end_vent_s)]
)
axis[1, 1].plot(
    x_emg[int(start_vent_s):int(end_vent_s)], 
    y_vent[1][int(start_vent_s):int(end_vent_s)]
)
axis[2, 1].plot(
    x_emg[int(start_vent_s):int(end_vent_s)], 
    y_vent[2][int(start_vent_s):int(end_vent_s)]
)

# Pre-processing

In [ ]:
# Eliminate the baseline wander from the data
bd_filtered_file_data = filt.emg_bandpass_butter_sample(
    data_emg_samples, 1, 500, 2048, output='sos')
    
processed_data_emg_di = bd_filtered_file_data[1]
processed_data_emg_para = bd_filtered_file_data[2]

In [ ]:
# Gating settings

# Window size to plot the gating results from
plot_window = 5         

# Peak height fraction
# peak_fraction = 0.30    
peak_fraction = gate_threshold_prev

# Shift of gated in samples relative to detected peaks
# ECG_shift = -10           
ECG_shift = gate_ECG_shift_prev

# Gate width in seconds
# gate_width = 0.1        
gate_width = gate_width_prev


# QRS peak detection settings
peak_width = 0.001
peak_dist = int(emg_fs/3)


In [ ]:
# Check number of gating iterations
if gate_twice == True:
    print('Signal will be gated TWICE!')
else:
    print('Signal will be gated ONCE!')

In [ ]:
# Apply QRS gating

# Detect ECG peaks on 1Hz high-pass filtered signals
ecg = bd_filtered_file_data[0]
ecg_rms = evl.full_rolling_rms(ecg, 10)
max_ecg_rms = max(
    ecg_rms[int(start_idx):int(start_idx+plot_window*emg_fs)])
min_ecg_rms = min(
    ecg_rms[int(start_idx):int(start_idx+plot_window*emg_fs)])
peak_height = peak_fraction*(max_ecg_rms - min_ecg_rms)

ECG_peaks, properties  = scipy.signal.find_peaks(
    ecg_rms, 
    height=peak_height, 
    width=peak_width*emg_fs, 
    distance=peak_dist
)
ECG_peaks = ECG_peaks - ECG_shift

# Filter EXG signals 20Hz high pass
ECG_h20 = filt.emg_bandpass_butter_sample(ecg, 20, 500, 2048, output='sos')
emg_di_h20 = filt.emg_bandpass_butter_sample(
    processed_data_emg_di, 20, 500, 2048, output='sos')
emg_para_h20 = filt.emg_bandpass_butter_sample(
    processed_data_emg_para, 20, 500, 2048, output='sos')

# Gate ECG and EMG signal
# Fill methods 0: Zeros, 1: Interpolate start-end, 2: Average prior data, 
# 3: Moving average
gate_fill_method = 3    
gate_width_samples = int(gate_width*emg_fs)

gate_samples = list()
for i in range(len(ECG_peaks)):
    for k in range(
        int(ECG_peaks[i]-gate_width_samples/2),
        int(ECG_peaks[i]+gate_width_samples/2)
    ):
        gate_samples.append(k)

ECG_gated = ecg_rm.gating(
    ECG_h20, ECG_peaks, gate_width=gate_width_samples, method=3)
emg_di_gated = ecg_rm.gating(
    emg_di_h20, ECG_peaks, gate_width=gate_width_samples, method=3)
emg_para_gated = ecg_rm.gating(
    emg_para_h20, ECG_peaks, gate_width=gate_width_samples, method=3)


In [ ]:
# Apply gating twice (in case of pacing)
if gate_twice == True:
    # Detect ECG peaks on 1Hz high-pass filtered signals
    ecg = ECG_gated
    ecg_rms = evl.full_rolling_rms(ecg, 10)
    max_ecg_rms = max(
        ecg_rms[int(start_idx):int(start_idx+plot_window*emg_fs)])
    min_ecg_rms = min(
        ecg_rms[int(start_idx):int(start_idx+plot_window*emg_fs)])
    peak_height = peak_fraction*(max_ecg_rms - min_ecg_rms)

    ECG_peaks, properties  = scipy.signal.find_peaks(
        ecg_rms, 
        height=peak_height, 
        width=peak_width*emg_fs, 
        distance=peak_dist
    )
    ECG_peaks = ECG_peaks - ECG_shift

    # Gate ECG and EMG signal
    # Fill methods: 0: Zeros, 1: Interpolate start-end, 2: Average prior data, 
    # 3: Moving average
    gate_fill_method = 3    
    gate_width_samples = int(gate_width*emg_fs)

    gate_samples = list()
    for i in range(len(ECG_peaks)):
        for k in range(
            int(ECG_peaks[i]-gate_width_samples/2),
            int(ECG_peaks[i]+gate_width_samples/2)
        ):
            gate_samples.append(k)

    ECG_gated = ecg_rm.gating(
        ECG_gated, ECG_peaks, gate_width=gate_width_samples, method=3)
    emg_di_gated = ecg_rm.gating(
        emg_di_gated, ECG_peaks, gate_width=gate_width_samples, method=3)
    emg_para_gated = ecg_rm.gating(
        emg_para_gated, ECG_peaks, gate_width=gate_width_samples, method=3)


In [ ]:
# Plot gating result

fig, axis = plt.subplots(nrows=5, ncols=1, figsize=(12, 6))
start_i = int(start_idx)
end_i = int(start_idx+plot_window*2048)
axis[0].grid(True)
axis[0].plot(x_emg[start_i:end_i], bd_filtered_file_data[0, start_i:end_i])
axis[0].set_ylabel('raw ECG (uV)')

axis[0].set(title='leads in EMG')
axis[1].grid(True)
axis[1].plot(x_emg[start_i:end_i], ECG_gated[start_i:end_i])
axis[1].set_ylabel('ECG gated (uV)')
axis[2].grid(True)
axis[2].plot(x_emg[start_i:end_i], emg_di_gated[start_i:end_i])
axis[2].set_ylabel('EMGdi gated (uV)')
axis[3].grid(True)
axis[3].plot(x_emg[start_i:end_i], emg_para_gated[start_i:end_i])
axis[3].set_ylabel('EMGpara gated (uV)')

axis[4].grid(True)
axis[4].plot(x_emg[start_i:end_i], ecg_rms[start_i:end_i])
axis[4].set_ylabel('ECG rms (uV)')
axis[4].hlines(
    y=peak_height, 
    xmin=x_emg[start_i], 
    xmax=x_emg[end_i],
    color = "C1"
)

for idx in range(len(ECG_peaks)):
    if ((ECG_peaks[idx] > int(start_idx)) 
        & (ECG_peaks[idx] < int(start_idx+plot_window*2048))):
        axis[4].plot(x_emg[ECG_peaks[idx]], ecg_rms[ECG_peaks[idx]],'rx')


Calculate root mean squared (RMS)

In [ ]:
RMS_window_ms = 200
RMS_windows_samp = int(RMS_window_ms / 1000 *  emg_fs)

RMS_data_emg_di = evl.full_rolling_rms(emg_di_gated, RMS_windows_samp)
RMS_data_emg_para = evl.full_rolling_rms(emg_para_gated, RMS_windows_samp)

fig, axis = plt.subplots(nrows=5, ncols=1, figsize=(12, 6))
axis[0].set(title='leads in EMG')
axis[0].set_ylabel('sEMGdi (uV)')
axis[0].grid(True)
axis[0].plot(x_emg[int(start_idx):int(end_idx)], 
             processed_data_emg_di[int(start_idx):int(end_idx)])

axis[0].plot(x_emg[int(start_idx):int(end_idx)], 
             RMS_data_emg_di[int(start_idx):int(end_idx)])

axis[1].set_xlabel('t (s)')
axis[1].set_ylabel('RMS{sEMGdi} (uV)')
axis[1].plot(x_emg[int(start_idx):int(end_idx)], 
             RMS_data_emg_di[int(start_idx):int(end_idx)])

axis[2].set(title='leads in EMG')
axis[2].set_ylabel('sEMGpara (uV)')
axis[2].grid(True)
axis[2].plot(x_emg[int(start_idx):int(end_idx)], 
             processed_data_emg_para[int(start_idx):int(end_idx)])
axis[2].plot(x_emg[int(start_idx):int(end_idx)], 
             RMS_data_emg_para[int(start_idx):int(end_idx)])

axis[3].set_xlabel('t (s)')
axis[3].set_ylabel('RMS{sEMGpara} (uV)')
axis[3].plot(x_emg[int(start_idx):int(end_idx)], 
             RMS_data_emg_para[int(start_idx):int(end_idx)])

axis[4].set_ylabel('P_vent (cmH2O)')
axis[4].grid(True)
axis[4].plot(x_emg[int(start_vent_s):int(end_vent_s)], 
             y_vent[0][int(start_vent_s):int(end_vent_s)])

# Feature extraction

In [ ]:
x = y_vent[2][:int(start_vent_s)]
V_ee_PKS, _ = scipy.signal.find_peaks(-x)

PEEP_set = np.floor(np.median(y_vent[0, V_ee_PKS]))


PEEP_set

In [ ]:
# Extract ECG-amplitude (RMS)
ecg_data_list_new = np.column_stack(
    (np.matlib.repmat([patient, measurement, PEEP_set], len(ECG_peaks), 1), 
     ecg_rms[ECG_peaks + ECG_shift])
)


In [ ]:
# Old Moving baseline QUICKer on pressure and all EMG signals for baseline 
# crossing detection

emg_baseline_factor = 1

baseline_W_emg = 5 * emg_fs  # window length
baseline_W_vent = 5 * vent_fs  # window length

emg_di_rolling_base_line_old = np.zeros(
    (len(RMS_data_emg_di[int(start_idx):int(end_idx)]), ))
emg_para_rolling_base_line_old = np.zeros(
    (len(RMS_data_emg_para[int(start_idx):int(end_idx)]), ))

for idx in range(0, int(end_idx)-int(start_idx), int(emg_fs/5)):
    start_i = max([int(start_idx), int(start_idx)+idx-int(baseline_W_emg/2)])
    end_i = min([int(end_idx), int(start_idx)+idx+int(baseline_W_emg/2)])
    baseline_value_emg_di = np.percentile(
        RMS_data_emg_di[start_i:end_i], 33)
    baseline_value_emg_para = np.percentile(
        RMS_data_emg_para[start_i:end_i], 33)
    for i in range(idx, 
                   min([idx+int(emg_fs/5), int(end_idx)-int(start_idx)])
    ):
        emg_di_rolling_base_line_old[i] = emg_baseline_factor * baseline_value_emg_di
        emg_para_rolling_base_line_old[i] = emg_baseline_factor * baseline_value_emg_para

In [ ]:
# Moving baseline QUICKer on pressure and all EMG signals for baseline crossing detection

emg_baseline_factor = 1.0
var_window_factor = int(1)

baseline_W_emg = int(7.5 * emg_fs)  # window length
baseline_W_vent = int(7.5 * vent_fs)  # window length

emg_di_rolling_base_line = np.zeros(
    (len(RMS_data_emg_di[int(start_idx):int(end_idx)]), ))
emg_para_rolling_base_line = np.zeros(
    (len(RMS_data_emg_para[int(start_idx):int(end_idx)]), ))
P_rolling_base_line = np.zeros(
    (len(y_vent[0, int(start_vent_s):int(end_vent_s)]), ))

for idx in range(0, int(end_idx)-int(start_idx), int(emg_fs/5)):
    start_i = max([int(start_idx), int(start_idx)+idx-int(baseline_W_emg/2)])
    end_i = min([int(end_idx), int(start_idx)+idx+int(baseline_W_emg/2)])
    baseline_value_emg_di = np.percentile(
        RMS_data_emg_di[start_i:end_i], 33)
    baseline_value_emg_para = np.percentile(
        RMS_data_emg_para[start_i:end_i], 33)
    for i in range(idx, 
                   min([idx+int(emg_fs/5), int(end_idx)-int(start_idx)])
    ):
        emg_di_rolling_base_line[i] = \
            emg_baseline_factor * baseline_value_emg_di
        emg_para_rolling_base_line[i] = \
            emg_baseline_factor * baseline_value_emg_para

di_baseline_series = pd.Series(emg_di_rolling_base_line)
di_baseline_std = di_baseline_series.rolling(var_window_factor*baseline_W_emg, 
                                 min_periods=1, 
                                 center=True).std().values
di_baseline_mean = di_baseline_series.rolling(var_window_factor*baseline_W_emg, 
                                 min_periods=1, 
                                 center=True).mean().values                                 
para_baseline_series = pd.Series(emg_para_rolling_base_line)
para_baseline_std = para_baseline_series.rolling(
    var_window_factor*baseline_W_emg, 
    min_periods=1, 
    center=True).std().values
para_baseline_mean = para_baseline_series.rolling(
    var_window_factor*baseline_W_emg, 
    min_periods=1, 
    center=True).mean().values

for idx in range(0, 
                 int(end_vent_s)-int(start_vent_s), 
                 int(vent_fs/5)
):
    start_i = max([int(start_vent_s), int(
        start_vent_s)+idx-int(baseline_W_vent/2)])
    end_i = min([int(end_vent_s), int(start_vent_s) +
                idx+int(baseline_W_vent/2)])
    P_rolling_base_line[idx] = np.percentile(y_vent[0, start_i:end_i], 33)

    baseline_value_P = np.percentile(y_vent[0, start_i:end_i], 33)
    for i in range(idx, 
            min([idx+int(vent_fs/5), 
                 int(end_vent_s)-int(start_vent_s)])
    ):
        P_rolling_base_line[i] = baseline_value_P

Get PTPocc

In [ ]:
# Pocc_prominence_factor = 0.3
Pocc_prominence_factor = Pocc_prominence_factor_prev

Pocc_prominence_factor

In [ ]:
# Detect (negative) occlussion peaks in pressure signal

treshold = 0
width = int(0.1 * vent_fs)
prominence = Pocc_prominence_factor *np.abs( PEEP_set - min(y_vent[0]))
height = -(PEEP_set - Pocc_prominence_factor * 
           np.abs( PEEP_set - min(y_vent[0])))
distance = int(0.5 * vent_fs)

x = y_vent[0, int(start_vent_s):int(end_vent_s)]
P_occ_peaks, properties  = scipy.signal.find_peaks(
    -x, 
    height=height, 
    prominence=prominence, 
    width=width, 
    distance=distance
)
# x = y_vent[0, int(start_vent_s):int(end_vent_s)]

PEEP_crossings_idx = np.argwhere(np.diff(np.sign(x - P_rolling_base_line)) != 0)

P_occ_starts = np.zeros((len(P_occ_peaks),), dtype=int)
P_occ_ends = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    PEEP_crossings_idx_sub = np.sign(
        x[:P_occ_peaks[idx]] - P_rolling_base_line[:P_occ_peaks[idx]])
    a = np.argmin(
        P_occ_peaks[idx] - np.argwhere(np.diff(PEEP_crossings_idx_sub) != 0))
    P_occ_starts[idx] = int(PEEP_crossings_idx[a])
    P_occ_ends[idx] = int(PEEP_crossings_idx[a+1])

P_occ_starts,P_occ_peaks,  P_occ_ends, prominence, height

In [ ]:
# Data sanity check: check to make sure your data has an occlusion even
if len(P_occ_starts) == 0 or len(P_occ_peaks) ==0  or len(P_occ_ends) == 0:
    print('You have chosen something without the proper sort of occlusion peak,' 
          + 'please examine data')
else:
    print('You may continue!')

EMG peak identification

In [ ]:
EMG_di_prominence_factor_prev, EMG_para_prominence_factor_prev

In [ ]:
# EMG peak detection parameters:

# Threshold peak height as fraction of max peak height 
# EMG_di_prominence_factor = 0.5         
EMG_di_prominence_factor = EMG_di_prominence_factor_prev

# Threshold peak height as fraction of max peak height
# EMG_para_prominence_factor = 0.5       
EMG_para_prominence_factor = EMG_para_prominence_factor_prev

# Draeger delay
# vent_delay = 0.5
vent_delay = time_shift_prev = 0.5                   

emg_peak_width = 0.2
percentile_border = 90

In [ ]:
# Augmented signal EMG + abs(dEMG/dt)
ma_window = window=emg_fs//2
augmented_perc = 25
perc_window = emg_fs

y_di = RMS_data_emg_di[int(start_idx):int(end_idx)]
s_di = pd.Series(y_di - emg_di_rolling_base_line)
EMG_di_MA = s_di.rolling(window=ma_window, center=True).mean().values
dEMG_di_dt = (EMG_di_MA[1:] - EMG_di_MA[:-1] ) * emg_fs
EMG_di_aug = y_di[:-1] + np.abs(dEMG_di_dt)

y_para = RMS_data_emg_para[int(start_idx):int(end_idx)]
s_para = pd.Series(y_para - emg_para_rolling_base_line)
EMG_para_MA = s_para.rolling(window=ma_window, center=True).mean().values
dEMG_para_dt = (EMG_para_MA[1:] - EMG_para_MA[:-1] ) * emg_fs
EMG_para_aug = y_para[:-1] + np.abs(dEMG_para_dt)

# Rolling baseline of augmented signals
emg_di_aug_rolling_base_line = np.zeros(
    (len(RMS_data_emg_di[int(start_idx):int(end_idx)-1]), ))
emg_para_aug_rolling_base_line = np.zeros(
    (len(RMS_data_emg_para[int(start_idx):int(end_idx)-1]), ))

for idx in range(0, int(end_idx-1)-int(start_idx), perc_window):
    start_i = max([0, idx-int(baseline_W_emg)])
    end_i = min([int(end_idx-start_idx-1), idx+int(baseline_W_emg)])

    baseline_value_emg_di = np.nanpercentile(
        EMG_di_aug[start_i:end_i], augmented_perc)

    baseline_value_emg_para = np.nanpercentile(
        EMG_para_aug[start_i:end_i], augmented_perc)
    
    for i in range(idx, 
                   min([idx+int(perc_window), int(end_idx-1)-int(start_idx)])
    ):
        emg_di_aug_rolling_base_line[i] = 1.2 * baseline_value_emg_di
        emg_para_aug_rolling_base_line[i] = 1.2 * baseline_value_emg_para

In [ ]:
# Find old baseline diaphragmatic EMG peaks

y_di = RMS_data_emg_di[int(start_idx):int(end_idx)]
treshold = 0
width = int(emg_peak_width * emg_fs)
prominence = EMG_di_prominence_factor * \
    (np.nanpercentile(y_di - emg_di_rolling_base_line_old, 75) 
     + np.nanpercentile(y_di - emg_di_rolling_base_line_old, 50))
EMG_peaks_di, properties = scipy.signal.find_peaks(
    y_di, height=treshold, prominence=prominence, width=width)

# Link EMG peak closest to occlusion pressures
EMG_di_occ_peaks = np.zeros((len(P_occ_peaks),), dtype=int)
PKS_idx = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    P_occ_peak_converted = P_occ_peaks[idx]/vent_fs*emg_fs
    PKS_idx[idx] = np.argmin(
        np.abs(EMG_peaks_di + vent_delay * emg_fs 
               - P_occ_peak_converted)
    )

    EMG_di_occ_peaks[idx] = EMG_peaks_di[PKS_idx[idx]]

# Old method
baseline_crossings_idx = np.argwhere(
    np.diff(np.sign(y_di - emg_di_rolling_base_line_old)) != 0)

EMG_di_occ_starts_old_base = np.zeros((len(P_occ_peaks),), dtype=int)
EMG_di_occ_ends_old_base = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    baseline_crossings_idx_sub = np.sign(
        y_di[:EMG_di_occ_peaks[idx]] 
        - emg_di_rolling_base_line_old[:EMG_di_occ_peaks[idx]]
    )
    a = np.argmin(
        EMG_di_occ_peaks[idx] 
        - np.argwhere(np.diff(baseline_crossings_idx_sub) != 0))
    
    EMG_di_occ_starts_old_base[idx] = int(baseline_crossings_idx[a])
    
    if a < len(baseline_crossings_idx) - 1:
        EMG_di_occ_ends_old_base[idx] = int(baseline_crossings_idx[a+1])
    else:
        EMG_di_occ_ends_old_base[idx] = len(y_di) - 1 

In [ ]:
# Find diaphragmatic EMG peaks

y_di = RMS_data_emg_di[int(start_idx):int(end_idx)]
treshold = 0
width = int(emg_peak_width * emg_fs)
prominence = EMG_di_prominence_factor * \
    (np.nanpercentile(y_di - emg_di_rolling_base_line, 75) 
     + np.nanpercentile(y_di - emg_di_rolling_base_line, 50))
EMG_peaks_di, properties = scipy.signal.find_peaks(
    y_di, height=treshold, prominence=prominence, width=width)

# Link EMG peak closest to occlusion pressures
EMG_di_occ_peaks = np.zeros((len(P_occ_peaks),), dtype=int)
PKS_idx = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    P_occ_peak_converted = P_occ_peaks[idx]/vent_fs*emg_fs
    PKS_idx[idx] = np.argmin(
        np.abs(EMG_peaks_di + vent_delay * emg_fs 
               - P_occ_peak_converted)
    )

    EMG_di_occ_peaks[idx] = EMG_peaks_di[PKS_idx[idx]]

emg_di_rolling_base_line_aug = emg_di_rolling_base_line * (1+ 
    np.nanmedian(y_di[EMG_di_occ_peaks]) * di_baseline_std 
    / di_baseline_mean **2
)
    

baseline_crossings_idx = np.argwhere(
    np.diff(np.sign(y_di - emg_di_rolling_base_line_aug)) != 0)

EMG_di_occ_starts_new = np.zeros((len(P_occ_peaks),), dtype=int)
EMG_di_occ_ends_new = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    baseline_crossings_idx_sub = np.sign(
        y_di[:EMG_di_occ_peaks[idx]] 
        - emg_di_rolling_base_line_aug[:EMG_di_occ_peaks[idx]]
    )
    a = np.argmin(
        EMG_di_occ_peaks[idx] 
        - np.argwhere(np.diff(baseline_crossings_idx_sub) != 0))
    
    EMG_di_occ_starts_new[idx] = int(baseline_crossings_idx[a])
    
    if a < len(baseline_crossings_idx) - 1:
        EMG_di_occ_ends_new[idx] = int(baseline_crossings_idx[a+1])
    else:
        EMG_di_occ_ends_new[idx] = len(y_di) - 1 

In [ ]:
# emg_di_base_factor

In [ ]:
# Find old baseline parasternal EMG peaks

y_para = RMS_data_emg_para[int(start_idx):int(end_idx)]
treshold = 0
width = int(emg_peak_width * emg_fs)
prominence = EMG_para_prominence_factor * \
    (np.nanpercentile(y_para - emg_para_rolling_base_line_old, 75) 
     + np.nanpercentile(y_para - emg_para_rolling_base_line_old, 50))

EMG_peaks_para, properties = scipy.signal.find_peaks(
    y_para, height=treshold, prominence=prominence, width=width)

# Link EMG peak closest to occlusion pressures
EMG_para_occ_peaks = np.zeros((len(P_occ_peaks),), dtype=int)
PKS_idx = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    P_occ_peak_converted = P_occ_peaks[idx]/vent_fs*emg_fs
    PKS_idx[idx] = np.argmin(
        np.abs(EMG_peaks_para + vent_delay * emg_fs 
               - P_occ_peak_converted))
    EMG_para_occ_peaks[idx] = EMG_peaks_para[PKS_idx[idx]]

# Old method
baseline_crossings_idx = np.argwhere(
    np.diff(np.sign(y_para - emg_para_rolling_base_line_old)) != 0)

EMG_para_occ_starts_old_base = np.zeros((len(P_occ_peaks),), dtype=int)
EMG_para_occ_ends_old_base = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    baseline_crossings_idx_sub = np.sign(
        y_para[:EMG_para_occ_peaks[idx]] 
        - emg_para_rolling_base_line_old[:EMG_para_occ_peaks[idx]])
    a = np.argmin(
        EMG_para_occ_peaks[idx] 
        - np.argwhere(np.diff(baseline_crossings_idx_sub) != 0))
    EMG_para_occ_starts_old_base[idx] = int(baseline_crossings_idx[a])
    if a < len(baseline_crossings_idx) - 1:
        EMG_para_occ_ends_old_base[idx] = int(baseline_crossings_idx[a+1])
    else:
        EMG_para_occ_ends_old_base[idx] = len(y_para) - 1 

EMG_para_occ_starts_old_base, EMG_para_occ_ends_old_base

In [ ]:
# Find parasternal EMG peaks

y_para = RMS_data_emg_para[int(start_idx):int(end_idx)]
treshold = 0
width = int(emg_peak_width * emg_fs)
prominence = EMG_para_prominence_factor * \
    (np.nanpercentile(y_para - emg_para_rolling_base_line, 75) 
     + np.nanpercentile(y_para - emg_para_rolling_base_line, 50))

EMG_peaks_para, properties = scipy.signal.find_peaks(
    y_para, height=treshold, prominence=prominence, width=width)

# Link EMG peak closest to occlusion pressures
EMG_para_occ_peaks = np.zeros((len(P_occ_peaks),), dtype=int)
PKS_idx = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    P_occ_peak_converted = P_occ_peaks[idx]/vent_fs*emg_fs
    PKS_idx[idx] = np.argmin(
        np.abs(EMG_peaks_para + vent_delay * emg_fs 
               - P_occ_peak_converted))
    EMG_para_occ_peaks[idx] = EMG_peaks_para[PKS_idx[idx]]

emg_para_rolling_base_line_aug = emg_para_rolling_base_line * (1+
    np.nanmedian(y_para[EMG_para_occ_peaks]) * para_baseline_std 
    / para_baseline_mean **2
)

baseline_crossings_idx = np.argwhere(
    np.diff(np.sign(y_para - emg_para_rolling_base_line_aug)) != 0)

EMG_para_occ_starts_new = np.zeros((len(P_occ_peaks),), dtype=int)
EMG_para_occ_ends_new = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    baseline_crossings_idx_sub = np.sign(
        y_para[:EMG_para_occ_peaks[idx]] 
        - emg_para_rolling_base_line_aug[:EMG_para_occ_peaks[idx]])
    
    a = np.argmin(
        EMG_para_occ_peaks[idx] 
        - np.argwhere(np.diff(baseline_crossings_idx_sub) != 0))
    EMG_para_occ_starts_new[idx] = int(baseline_crossings_idx[a])
    if a < len(baseline_crossings_idx) - 1:
        EMG_para_occ_ends_new[idx] = int(baseline_crossings_idx[a+1])
    else:
        EMG_para_occ_ends_new[idx] = len(y_para) - 1 

In [ ]:
emg_di_base_factor = 1
emg_para_base_factor = 1

In [ ]:
# Plot new baseline crossing detection diaphragm
fig_5, axis = plt.subplots(nrows=3, ncols=max([len(
    P_occ_peaks),2]), figsize=(12, 6), sharex='col', sharey='row')

# Show EMG and Paw data for entire window
t_plot_start = np.min([[P_occ_starts / vent_fs], 
                    [EMG_di_occ_starts_new / emg_fs],
                    [EMG_di_occ_starts_old_base / emg_fs], 
                    [EMG_para_occ_starts_new / emg_fs]], axis=0)[0] - 0.5
t_plot_end = np.max([[P_occ_ends / vent_fs], 
                     [EMG_di_occ_ends_new / emg_fs], 
                     [EMG_di_occ_ends_old_base / emg_fs], 
                     [EMG_para_occ_ends_new / emg_fs]], axis=0)[0] + 0.5

PTP_occs = np.zeros((len(P_occ_peaks),))
p_vent_max = np.zeros((len(P_occ_peaks),))
PTP_occ_baseline = np.zeros((len(P_occ_peaks),))
for idx in range(len(P_occ_peaks)):
    start_i = P_occ_starts[idx]
    end_i = P_occ_ends[idx]
    start_plot_i = max([int(t_plot_start[idx] * vent_fs), 0])
    end_plot_i = min(
        [int(t_plot_end[idx] * vent_fs), len(x)-1])

    axis[0, idx].grid(True)
    axis[0, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                      x[start_plot_i:end_plot_i+1])
    axis[0, idx].plot(x_emg[start_plot_i:end_plot_i+1],
                      P_rolling_base_line[start_plot_i:end_plot_i+1])    

    baseline_start_i = max([0, P_occ_peaks[idx] - 5*vent_fs])
    baseline_end_i = min([len(x) - 1, P_occ_peaks[idx] + 5*vent_fs])
    p_vent_max[idx] = max(P_rolling_base_line[baseline_start_i:baseline_end_i])

    axis[0, idx].plot([x_emg[start_i], x_emg[end_i+1]], 
                      [p_vent_max[idx], p_vent_max[idx]], 'c')
    axis[0, idx].plot([x_emg[start_i], x_emg[start_i]], 
                      [x[start_i], p_vent_max[idx]], 'c')
    axis[0, idx].plot([x_emg[end_i], x_emg[end_i]], 
                      [x[end_i], p_vent_max[idx]], 'c')


    axis[0, idx].plot(x_emg[P_occ_peaks[idx]], x[P_occ_peaks[idx]], '*r')
    axis[0, idx].plot(x_emg[start_i], x[start_i], '*r')
    axis[0, idx].plot(x_emg[end_i], x[end_i], '*r')

axis[0, 0].set_ylabel('Pocc (cmH2O)')


y_di_min = np.zeros((len(EMG_di_occ_peaks),))
for idx in range(len(EMG_di_occ_peaks)):
    start_i = EMG_di_occ_starts_new[idx]
    end_i = EMG_di_occ_ends_new[idx]
    start_plot_i = max([int(t_plot_start[idx] * emg_fs), 0])
    end_plot_i = min([int(t_plot_end[idx] * emg_fs), len(y_di)-1])
    
    axis[1, idx].grid(True)
    
    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                      y_di[start_plot_i:end_plot_i+1])
    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i+1],
                      emg_di_base_factor*
                      emg_di_rolling_base_line_aug[start_plot_i:end_plot_i+1])
    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i+1],
                      emg_di_rolling_base_line_old[start_plot_i:end_plot_i+1], 'k')

    baseline_start_i = max([0, EMG_di_occ_peaks[idx] - 5*emg_fs])
    baseline_end_i = min(
        [len(y_di) - 1, EMG_di_occ_peaks[idx] + 5*emg_fs])
    y_di_min[idx] = min(y_di[baseline_start_i:baseline_end_i])
    
    axis[1, idx].set_xlabel('t (s)')
    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i], 
                      EMG_di_aug[start_plot_i:end_plot_i], 
                      'tab:gray', linewidth=0.5)
    
    axis[2, idx].grid(True)
    axis[2, idx].plot(x_emg[start_plot_i:end_plot_i], 
                      EMG_di_aug[start_plot_i:end_plot_i])
    axis[2, idx].plot(x_emg[start_plot_i:end_plot_i], 
                      emg_di_aug_rolling_base_line[start_plot_i:end_plot_i])
    axis[2, idx].set_xlabel('t (s)')
    
    axis[1, idx].plot(x_emg[EMG_di_occ_starts_old_base[idx]], 
                      y_di[EMG_di_occ_starts_old_base[idx]], '*g')
    axis[1, idx].plot(x_emg[EMG_di_occ_ends_old_base[idx]], 
                      y_di[EMG_di_occ_ends_old_base[idx]], '*g')
    axis[1, idx].plot(x_emg[EMG_di_occ_starts_new[idx]-ma_window//2], 
                        y_di[EMG_di_occ_starts_new[idx]-ma_window//2], '*k')
    axis[1, idx].plot(x_emg[EMG_di_occ_ends_new[idx]+ma_window//2], 
                        y_di[EMG_di_occ_ends_new[idx]+ma_window//2], '*k')
    axis[1, idx].plot(x_emg[EMG_di_occ_peaks[idx]],
                      y_di[EMG_di_occ_peaks[idx]], '*r')
    axis[1, idx].plot(x_emg[start_i], y_di[start_i], '*r')
    axis[1, idx].plot(x_emg[end_i], y_di[end_i], '*r')
    
                

axis[1, 0].set_ylabel('sEMGdi (uV)')
axis[2, 0].set_ylabel('EMGdi + abs(dEMGdi/dt) (.)')
print(EMG_di_occ_starts_old_base,
      EMG_di_occ_starts_new,
      EMG_di_occ_ends_old_base,
      EMG_di_occ_ends_new)



In [ ]:
# Plot new baseline crossing detection parasternal
fig_6, axis = plt.subplots(nrows=3, ncols=max([len(
    P_occ_peaks),2]), figsize=(12, 6), sharex='col', sharey='row')

# Show EMG and Paw data for entire window
t_plot_start = np.min(
    [[P_occ_starts / vent_fs], 
    [EMG_di_occ_starts_new / emg_fs], 
    [EMG_para_occ_starts_old_base / emg_fs], 
    [EMG_para_occ_starts_new / emg_fs]], axis=0)[0] - 0.5
t_plot_end = np.max(
    [[P_occ_ends / vent_fs], 
    [EMG_di_occ_ends_new / emg_fs], 
    [EMG_para_occ_ends_old_base / emg_fs], 
    [EMG_para_occ_ends_new / emg_fs]], axis=0)[0] + 0.5

PTP_occs = np.zeros((len(P_occ_peaks),))
p_vent_max = np.zeros((len(P_occ_peaks),))
PTP_occ_baseline = np.zeros((len(P_occ_peaks),))
for idx in range(len(P_occ_peaks)):
    start_i = P_occ_starts[idx]
    end_i = P_occ_ends[idx]
    start_plot_i = max([int(t_plot_start[idx] * vent_fs), 0])
    end_plot_i = min(
        [int(t_plot_end[idx] * vent_fs), len(x)-1])

    axis[0, idx].grid(True)
    axis[0, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                      x[start_plot_i:end_plot_i+1])
    axis[0, idx].plot(x_emg[start_plot_i:end_plot_i+1],
                      P_rolling_base_line[start_plot_i:end_plot_i+1])    

    baseline_start_i = max([0, P_occ_peaks[idx] - 5*vent_fs])
    baseline_end_i = min([len(x) - 1, P_occ_peaks[idx] + 5*vent_fs])
    p_vent_max[idx] = max(P_rolling_base_line[baseline_start_i:baseline_end_i])

    axis[0, idx].plot([x_emg[start_i], x_emg[end_i+1]], 
                      [p_vent_max[idx], p_vent_max[idx]], 'c')
    axis[0, idx].plot([x_emg[start_i], x_emg[start_i]], 
                      [x[start_i], p_vent_max[idx]], 'c')
    axis[0, idx].plot([x_emg[end_i], x_emg[end_i]], 
                      [x[end_i], p_vent_max[idx]], 'c')


    axis[0, idx].plot(x_emg[P_occ_peaks[idx]], x[P_occ_peaks[idx]], '*r')
    axis[0, idx].plot(x_emg[start_i], x[start_i], '*r')
    axis[0, idx].plot(x_emg[end_i], x[end_i], '*r')

axis[0, 0].set_ylabel('Pocc (cmH2O)')


y_para_min = np.zeros((len(EMG_para_occ_peaks),))
for idx in range(len(EMG_para_occ_peaks)):
    start_i = EMG_para_occ_starts_new[idx]
    end_i = EMG_para_occ_ends_new[idx]
    start_plot_i = max([int(t_plot_start[idx] * emg_fs), 0])
    end_plot_i = min([int(t_plot_end[idx] * emg_fs), len(y_para)-1])
    
    axis[1, idx].grid(True)
    
    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                      y_para[start_plot_i:end_plot_i+1])
    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i+1],
                      emg_para_base_factor *
                      emg_para_rolling_base_line_aug[start_plot_i:end_plot_i+1])
    axis[1, idx].plot(
        x_emg[start_plot_i:end_plot_i+1],
        emg_para_rolling_base_line_old[start_plot_i:end_plot_i+1], 'k')

    baseline_start_i = max([0, EMG_para_occ_peaks[idx] - 5*emg_fs])
    baseline_end_i = min(
        [len(y_para) - 1, EMG_para_occ_peaks[idx] + 5*emg_fs])
    y_para_min[idx] = min(y_para[baseline_start_i:baseline_end_i])

    axis[1, idx].plot(x_emg[EMG_para_occ_peaks[idx]],
                      y_para[EMG_para_occ_peaks[idx]], '*r')
    axis[1, idx].plot(x_emg[start_i], y_para[start_i], '*r')
    axis[1, idx].plot(x_emg[end_i], y_para[end_i], '*r')
    
    axis[1, idx].set_xlabel('t (s)')
    
    axis[2, idx].grid(True)
    axis[2, idx].plot(x_emg[start_plot_i:end_plot_i], 
                      EMG_para_aug[start_plot_i:end_plot_i])
    axis[2, idx].plot(x_emg[start_plot_i:end_plot_i], 
                      emg_para_aug_rolling_base_line[start_plot_i:end_plot_i])
    axis[2, idx].set_xlabel('t (s)')
    
    axis[1, idx].plot(x_emg[EMG_para_occ_starts_old_base[idx]], 
                      y_para[EMG_para_occ_starts_old_base[idx]], '*g')
    axis[1, idx].plot(x_emg[EMG_para_occ_ends_old_base[idx]], 
                      y_para[EMG_para_occ_ends_old_base[idx]], '*g')
    axis[1, idx].plot(x_emg[EMG_para_occ_starts_new[idx]-ma_window//2], 
                        y_para[EMG_para_occ_starts_new[idx]-ma_window//2], '*k')
    axis[1, idx].plot(x_emg[EMG_para_occ_ends_new[idx]+ma_window//2], 
                        y_para[EMG_para_occ_ends_new[idx]+ma_window//2], '*k')
    
                
axis[1, 0].set_ylabel('sEMGpara (uV)')
axis[2, 0].set_ylabel('EMGpara + abs(dEMGpara/dt) (.)')

print(
    EMG_para_occ_starts_old_base,
    EMG_para_occ_starts_new,
    EMG_para_occ_ends_old_base,
    EMG_para_occ_ends_new
)

In [ ]:
# Determine interpeak distance -> ECG or EMG?
# Determine pneumatic interpeak distance -> Using pressure signal
P_abs_t = np.abs(x - PEEP_set)
treshold = 0.5 * np.percentile(P_abs_t, 90)
prominence = 0.5 * np.percentile(P_abs_t, 90)
width = 0.25 * vent_fs
resp_efforts, _ = scipy.signal.find_peaks(
    P_abs_t, height=treshold, prominence=prominence, width=width)

t_delta_resp_efforts = np.median(np.array(x_emg)[resp_efforts[1:]] 
                                 - np.array(x_emg)[resp_efforts[:-1]])


# Determine pneumatic interpeak distance -> Using volume signal
V_t = y_vent[2, int(start_vent_s):int(end_vent_s)]
treshold = 0.25 * np.percentile(V_t, 90)
prominence = 0.10 * np.percentile(V_t, 90)
width = 0.25 * vent_fs
resp_efforts_volume_1, _ = scipy.signal.find_peaks(
    V_t, height=treshold, prominence=prominence, width=width)

treshold = 0.5 * np.percentile(V_t[resp_efforts_volume_1], 90)
prominence = 0.5 * np.percentile(V_t, 90)
width = 0.25 * vent_fs
resp_efforts_volume, _ = scipy.signal.find_peaks(
    V_t, height=treshold, prominence=prominence, width=width)

t_delta_resp_efforts_volume = np.median(
    np.array(x_emg)[resp_efforts_volume[1:]] 
    - np.array(x_emg)[resp_efforts_volume[:-1]])

# Determine electrophysiological interpeak distance
t_delta_di_med = np.median(np.array(x_emg)[EMG_peaks_di[1:]] 
                           - np.array(x_emg)[EMG_peaks_di[:-1]])
t_delta_para_med = np.median(np.array(x_emg)[EMG_peaks_para[1:]] 
                             - np.array(x_emg)[EMG_peaks_para[:-1]])
t_delta_ecg_med = np.median(np.array(x_emg)[ECG_peaks[1:]] 
                            - np.array(x_emg)[ECG_peaks[:-1]])


# t_delta_resp_efforts, t_delta_di_med, t_delta_para_med, t_delta_ecg_med
t_delta_resp_efforts, t_delta_resp_efforts_volume

In [ ]:
# Plot entire selected time window with indicated peaks
# P_vent
fig_1, axis = plt.subplots(nrows=4, ncols=1, figsize=(12, 6))
axis[0].grid(True)
axis[0].plot([y / vent_fs for y in range(len(x))], x)
axis[0].plot([y / vent_fs for y in range(len(x))],
             P_rolling_base_line)
axis[0].plot(P_occ_peaks/vent_fs, x[P_occ_peaks], "x", color="r")
axis[0].plot(P_occ_starts / vent_fs,
             P_rolling_base_line[P_occ_starts], '*r')
axis[0].plot(P_occ_ends / vent_fs,
             P_rolling_base_line[P_occ_ends], '*r')
axis[0].set_ylabel('P_vent (cmH2O)')

# V_vent
y_vent = y_vent[2, int(start_vent_s):int(end_vent_s)]
axis[1].grid(True)
axis[1].plot([y / vent_fs for y in range(len(x))], y_vent)
axis[1].set_ylabel('V_vent (mL)')

double_dips = np.zeros((len(P_occ_peaks),))
for idx in range(len(P_occ_peaks)):
    if idx > 0 :
        intermediate_breaths = np.equal(
            (P_occ_peaks[idx-1] < resp_efforts_volume  ), 
            (resp_efforts_volume < P_occ_peaks[idx]))
        
        if len(intermediate_breaths[intermediate_breaths == True]) > 0:
            double_dips[idx] = False
        else:
            double_dips[idx] = True

        axis[1].plot(
            resp_efforts_volume[intermediate_breaths]/vent_fs, 
            y_vent[resp_efforts_volume[intermediate_breaths]], "x", color="g")

    else:
        double_dips[idx] = False

# sEMGdi:
axis[2].grid(True)
N_samp = len(RMS_data_emg_di[int(start_idx):int(end_idx)])
axis[2].plot([y / emg_fs for y in range(N_samp)],
             RMS_data_emg_di[int(start_idx):int(end_idx)])
axis[2].plot([y / emg_fs for y in range(N_samp)],
             emg_di_base_factor*emg_di_rolling_base_line_aug[:int(end_idx)-int(start_idx)])
axis[2].plot(EMG_di_occ_peaks/emg_fs,
             y_di[EMG_di_occ_peaks], "x", color="r")
axis[2].plot(EMG_di_occ_starts_new/ emg_fs,
             y_di[EMG_di_occ_starts_new], '*r')
axis[2].plot(EMG_di_occ_ends_new / emg_fs,
             y_di[EMG_di_occ_ends_new], '*r')
axis[2].set_ylabel('RMS{sEMGdi} (uV)')
axis[2].set_xlabel('t (s)')
axis[2].set_ylim([0, 1.2*max(y_di[EMG_di_occ_peaks])])

# sEMGopara
axis[3].grid(True)
axis[3].plot([y / emg_fs for y in range(N_samp)],
             RMS_data_emg_para[int(start_idx):int(end_idx)])
axis[3].plot(EMG_para_occ_peaks/emg_fs,
             y_para[EMG_para_occ_peaks], "x", color="r")

axis[3].plot([y / emg_fs for y in range(N_samp)],
             emg_para_base_factor*emg_para_rolling_base_line_aug[:int(end_idx)-int(start_idx)])
axis[3].plot(EMG_para_occ_starts_new / emg_fs,
             y_para[EMG_para_occ_starts_new], '*r')
axis[3].plot(EMG_para_occ_ends_new / emg_fs,
             y_para[EMG_para_occ_ends_new], '*r')
axis[3].set_ylabel('RMS{sEMGpara} (uV)')
axis[3].set_xlabel('t (s)')
axis[3].set_ylim([0, 1.2*max(y_para[EMG_para_occ_peaks])])

In [ ]:
# Subplot per indicated peak
fig_2, axis = plt.subplots(nrows=3, ncols=max([len(
    P_occ_peaks),2]), figsize=(12, 6), sharex='col', sharey='row')

# Show EMG and Paw data for entire window
t_plot_start = np.min([[P_occ_starts / vent_fs], 
                    [EMG_di_occ_starts_new / emg_fs], 
                    [EMG_para_occ_starts_new / emg_fs]], axis=0)[0] - 0.5
t_plot_end = np.max([[P_occ_ends / vent_fs], 
                     [EMG_di_occ_ends_new / emg_fs], 
                     [EMG_para_occ_ends_new / emg_fs]], axis=0)[0] + 0.5

PTP_occs = np.zeros((len(P_occ_peaks),))
P_peak_occs = np.zeros((len(P_occ_peaks),))
p_vent_max = np.zeros((len(P_occ_peaks),))
PTP_occ_baseline = np.zeros((len(P_occ_peaks),))
PTP_occs_peak = np.zeros((len(P_occ_peaks),))
PTP_occ_peak_baseline = np.zeros((len(P_occ_peaks),))

for idx in range(len(P_occ_peaks)):
    start_i = P_occ_starts[idx]
    end_i = P_occ_ends[idx]
    start_plot_i = max([int(t_plot_start[idx] * vent_fs), 0])
    end_plot_i = min(
        [int(t_plot_end[idx] * vent_fs), len(x_emg)])

    axis[0, idx].grid(True)
    axis[0, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                      x[start_plot_i:end_plot_i+1])
    axis[0, idx].plot(x_emg[start_plot_i:end_plot_i+1],
                      P_rolling_base_line[start_plot_i:end_plot_i+1])    

    baseline_start_i = max([0, P_occ_peaks[idx] - 5*vent_fs])
    baseline_end_i = min([len(x) - 1, P_occ_peaks[idx] + 5*vent_fs])
    p_vent_max[idx] = max(P_rolling_base_line[baseline_start_i:baseline_end_i])

    axis[0, idx].plot([x_emg[start_i], x_emg[end_i+1]], 
                      [p_vent_max[idx], p_vent_max[idx]], 'c')
    axis[0, idx].plot([x_emg[start_i], x_emg[start_i]], 
                      [x[start_i], p_vent_max[idx]], 'c')
    axis[0, idx].plot([x_emg[end_i], x_emg[end_i]], 
                      [x[end_i], p_vent_max[idx]], 'c')

    P_peak_occs[idx] = x[P_occ_peaks[idx]]
    
    PTP_occ_baseline[idx] = np.trapz(
        p_vent_max[idx] - P_rolling_base_line[start_i:end_i+1],
        dx=1/vent_fs
    )

    PTP_occ_peak_baseline[idx] = np.trapz(
        p_vent_max[idx] - P_rolling_base_line[start_i:P_occ_peaks[idx]+1],
        dx=1/vent_fs
    )

    axis[0, idx].plot(x_emg[P_occ_peaks[idx]], x[P_occ_peaks[idx]], '*r')
    axis[0, idx].plot(x_emg[start_i], x[start_i], '*r')
    axis[0, idx].plot(x_emg[end_i], x[end_i], '*r')

    # Pressure Time Product (ETP) occlusion
    PTP_occs[idx] = np.abs(np.trapz(
        x[start_i:end_i+1]-P_rolling_base_line[start_i:end_i+1],
        dx=1/vent_fs
    )) + PTP_occ_baseline[idx]

    PTP_occs_peak[idx] = np.abs(np.trapz(
        x[start_i:P_occ_peaks[idx]+1]
        - P_rolling_base_line[start_i:P_occ_peaks[idx]+1],
        dx=1/vent_fs
    )) + PTP_occ_peak_baseline[idx]

axis[0, 0].set_ylabel('Pocc (cmH2O)')

ETP_di_occs = np.zeros((len(EMG_di_occ_peaks),))
y_di_min = np.zeros((len(EMG_di_occ_peaks),))
ETP_di_baseline = np.zeros((len(EMG_di_occ_peaks),))
ETP_di_baseline_old = np.zeros((len(EMG_di_occ_peaks),))
for idx in range(len(EMG_di_occ_peaks)):
    start_i = EMG_di_occ_starts_new[idx]
    end_i = EMG_di_occ_ends_new[idx]
    start_plot_i = max([int(t_plot_start[idx] * emg_fs), 0])
    end_plot_i = min([int(t_plot_end[idx] * emg_fs), len(x_emg)])
    
    axis[1, idx].grid(True)
    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i], 
                      ecg_rms[start_plot_i+205:end_plot_i+205]/
                      max(ecg_rms[start_plot_i+205:end_plot_i+205])*
                      max(y_di[start_plot_i:end_i]), 
                      'tab:gray', linewidth=0.5)
    
    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                      y_di[start_plot_i:end_plot_i+1])
    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i+1],
                      emg_di_rolling_base_line[start_plot_i:end_plot_i+1])

    baseline_start_i = max([0, EMG_di_occ_peaks[idx] - 5*emg_fs])
    baseline_end_i = min(
        [len(y_di) - 1, EMG_di_occ_peaks[idx] + 5*emg_fs])
    y_di_min[idx] = min(y_di[baseline_start_i:baseline_end_i])

    axis[1, idx].plot([x_emg[start_i], x_emg[end_i+1]], 
                      [y_di_min[idx], y_di_min[idx]], 'c')
    axis[1, idx].plot([x_emg[start_i], x_emg[start_i]], 
                      [y_di[start_i], y_di_min[idx]], 'c')
    axis[1, idx].plot([x_emg[end_i], x_emg[end_i]], 
                      [y_di[end_i], y_di_min[idx]], 'c')

    ETP_di_baseline[idx] = np.trapz(
        emg_di_base_factor*emg_di_rolling_base_line_aug[start_i:end_i+1] 
        - y_di_min[idx], dx=1/emg_fs
    )

    ETP_di_baseline_old[idx] = np.trapz(
        emg_di_base_factor*emg_di_rolling_base_line[start_i:end_i+1] - y_di_min[idx],
        dx=1/emg_fs
    )

    axis[1, idx].plot(x_emg[EMG_di_occ_peaks[idx]],
                      y_di[EMG_di_occ_peaks[idx]], '*r')
    axis[1, idx].plot(x_emg[start_i], y_di[start_i], '*r')
    axis[1, idx].plot(x_emg[end_i], y_di[end_i], '*r')


    
    axis[1, idx].set_ylim([0, 1.2*max(y_di[EMG_di_occ_peaks])])

    # EMG Time Product (ETP) diaphragm
    ETP_di_occs[idx] = np.trapz(
        y_di[start_i:end_i+1] - emg_di_base_factor * 
        emg_di_rolling_base_line_aug[start_i:end_i+1],
        dx=1/emg_fs
    ) + ETP_di_baseline[idx]

    axis[1, idx].set_xlabel('t (s)')

axis[1, 0].set_ylabel('sEMGdi (uV)')

ETP_para_occs = np.zeros((len(EMG_para_occ_peaks),))
y_para_min = np.zeros((len(EMG_para_occ_peaks),))
ETP_para_baseline = np.zeros((len(EMG_para_occ_peaks),))
ETP_para_baseline_old = np.zeros((len(EMG_para_occ_peaks),))
for idx in range(len(EMG_para_occ_peaks)):
    start_i = EMG_para_occ_starts_new[idx]
    end_i = EMG_para_occ_ends_new[idx]
    start_plot_i = max([int(t_plot_start[idx] * emg_fs), 0])
    end_plot_i = min([int(t_plot_end[idx] * emg_fs), len(x_emg)])

    axis[2, idx].grid(True)
    axis[2, idx].plot(x_emg[start_plot_i:end_plot_i], 
                    ecg_rms[start_plot_i+205:end_plot_i+205]/
                    max(ecg_rms[start_plot_i+205:end_plot_i+205])*
                    max(y_para[start_plot_i:end_plot_i]), 
                    'tab:gray', linewidth=0.5)
    axis[2, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                      y_para[start_plot_i:end_plot_i+1])
    axis[2, idx].plot(x_emg[start_plot_i:end_plot_i+1],
                      emg_para_rolling_base_line[start_plot_i:end_plot_i+1])
    
    baseline_start_i = max([0, EMG_para_occ_peaks[idx] - 5*emg_fs])
    baseline_end_i = min(
        [len(y_para) - 1, EMG_para_occ_peaks[idx] + 5*emg_fs])
    y_para_min[idx] = min(y_para[baseline_start_i:baseline_end_i])

    axis[2, idx].plot([x_emg[start_i], x_emg[end_i+1]], 
                      [y_para_min[idx], y_para_min[idx]], 'c')
    axis[2, idx].plot([x_emg[start_i], x_emg[start_i]], 
                      [y_para[start_i], y_para_min[idx]], 'c')
    axis[2, idx].plot([x_emg[end_i], x_emg[end_i]], 
                      [y_para[end_i], y_para_min[idx]], 'c')

    ETP_para_baseline[idx] = np.trapz(emg_para_base_factor*
        emg_para_rolling_base_line_aug[start_i:end_i+1] - y_para_min[idx],
        dx=1/emg_fs
    )

    ETP_para_baseline_old[idx] = np.trapz(
        emg_para_rolling_base_line[start_i:end_i+1] - y_para_min[idx],
        dx=1/emg_fs
    )

    axis[2, idx].plot(x_emg[EMG_para_occ_peaks[idx]],
                      y_para[EMG_para_occ_peaks[idx]], '*r')
    axis[2, idx].plot(x_emg[start_i], y_para[start_i], '*r')
    axis[2, idx].plot(x_emg[end_i], y_para[end_i], '*r')

    # EMG Time Product (ETP) parasternal
    ETP_para_occs[idx] = np.trapz(y_para[start_i:end_i+1] 
        - emg_para_base_factor *emg_para_rolling_base_line_aug[start_i:end_i+1],
        dx=1/emg_fs
    ) + ETP_para_baseline[idx]

    axis[2, idx].set_xlabel('t (s)')

axis[2, 0].set_ylabel('sEMGpara (uV)')


# Calculated neuromuscular efficiency: NME = PTP / ETP_di
NME_di = np.abs(PTP_occs)/ETP_di_occs

SNR_di = feat.snr_pseudo(
    src_signal=RMS_data_emg_di[int(start_idx):int(end_idx)], 
    peaks=EMG_di_occ_peaks, 
    baseline=emg_di_base_factor*
        emg_di_rolling_base_line_aug[:int(end_idx)-int(start_idx)]\
)
SNR_para = feat.snr_pseudo(
    src_signal=RMS_data_emg_para[int(start_idx):int(end_idx)], 
    peaks=EMG_para_occ_peaks, 
    baseline= emg_para_base_factor*
        emg_para_rolling_base_line_aug[:int(end_idx)-int(start_idx)]    
)


PTP_occs, ETP_di_occs, ETP_para_occs, NME_di, SNR_di, SNR_para

In [ ]:
# dPocc in up and downslope comparison for early release detection

fig_4, axis_tmp = plt.subplots(nrows=2, ncols=max([len(
    P_occ_peaks),2]), figsize=(12, 6), sharex='row', sharey='row')

x_emg = np.array(x_emg)

n_bins = 20
# PTP_occs
dP_up_10 = np.zeros((len(P_occ_peaks),))
dP_up_90 = np.zeros((len(P_occ_peaks),))
for idx in range(len(P_occ_peaks)):

    start_i = P_occ_starts[idx]
    end_i = P_occ_ends[idx]
    start_plot_i = max([int(t_plot_start[idx] * vent_fs), 0])
    end_plot_i = min(
        [int(t_plot_end[idx] * vent_fs), len(x)-1])
    
    # P_vent-time plots
    axis_tmp[0, idx].grid(True)
    axis_tmp[0, idx].plot(
        x_emg[start_plot_i:end_plot_i+1] - x_emg[start_plot_i], 
        x[start_plot_i:end_plot_i+1])
    axis_tmp[0, idx].plot(
        x_emg[start_plot_i:end_plot_i+1]- x_emg[start_plot_i],
        P_rolling_base_line[start_plot_i:end_plot_i+1])    

    baseline_start_i = max([0, P_occ_peaks[idx] - 5*vent_fs])
    baseline_end_i = min([len(x) - 1, P_occ_peaks[idx] + 5*vent_fs])
    p_vent_max[idx] = max(P_rolling_base_line[baseline_start_i:baseline_end_i])

    axis_tmp[0, idx].plot(
        [x_emg[start_i], x_emg[end_i+1]] - x_emg[start_plot_i], 
        [p_vent_max[idx], p_vent_max[idx]], 'c')
    axis_tmp[0, idx].plot(
        [x_emg[start_i], x_emg[start_i]] - x_emg[start_plot_i], 
        [x[start_i], p_vent_max[idx]], 'c')
    axis_tmp[0, idx].plot(
        [x_emg[end_i], x_emg[end_i]] - x_emg[start_plot_i], 
        [x[end_i], p_vent_max[idx]], 'c')

    axis_tmp[0, idx].plot(x_emg[P_occ_peaks[idx]] - x_emg[start_plot_i],
                    x[P_occ_peaks[idx]], '*r')
    axis_tmp[0, idx].plot(x_emg[start_i] - x_emg[start_plot_i],
                      x[start_i], '*r')
    axis_tmp[0, idx].plot(x_emg[end_i] - x_emg[start_plot_i], 
                      x[end_i], '*r')
    axis_tmp[0, idx].set_xlabel('t (s)')

    dP_up_10[idx] = np.percentile(
        x[P_occ_peaks[idx]+1:end_i]-x[P_occ_peaks[idx]:end_i-1], 10)
    dP_up_90[idx] = np.percentile(
        x[P_occ_peaks[idx]+1:end_i]-x[P_occ_peaks[idx]:end_i-1], 90)
    
    # Histogram of dPocc/dx in down- (blue) and upslope (red)
    axis_tmp[1, idx].grid(True)
    axis_tmp[1, idx].hist(
        x[start_i+1:P_occ_peaks[idx]+1]-x[start_i:P_occ_peaks[idx]], 
        bins=n_bins, color='b')

    axis_tmp[1, idx].grid(True)
    axis_tmp[1, idx].hist(
        x[P_occ_peaks[idx]+1:end_i]-x[P_occ_peaks[idx]:end_i-1], 
        bins=n_bins, color='r')

    axis_tmp[1, idx].set_xlabel('dPocc (cmH2O)')

# Make x-axes symmetric in y-axis
low, high = axis_tmp[1, 0].get_xlim()
bound = max(abs(low), abs(high))
axis_tmp[1, 0].set_xlim(-bound, bound)
axis_tmp[0, 0].set_ylabel('Pocc (cmH2O)')
axis_tmp[1, 0].set_ylabel('Sample count (.)')


In [ ]:
# Fit to bell plots
fig_3, axis = plt.subplots(nrows=3, ncols=max([len(
    P_occ_peaks), 2]), figsize=(12, 6), sharex='col', sharey='row')

# Show EMG and Paw data for entire window
t_plot_start = np.min([[P_occ_starts / vent_fs], 
                    [EMG_di_occ_starts_new/ emg_fs], 
                    [EMG_para_occ_starts_new / emg_fs]], axis=0)[0] - 0.5
t_plot_end = np.max([[P_occ_ends / vent_fs], 
                     [EMG_di_occ_ends_new / emg_fs], 
                     [EMG_para_occ_ends_new / emg_fs]], axis=0)[0] + 0.5

from scipy.optimize import curve_fit

def func(x, a, b, c):
    return a * np.exp(-(x-b)**2 / c**2)

# PTP_occ_baseline[idx], ETP_di_baseline[idx], ETP_para_baseline[idx]]
PTP_occ_bell_error = np.zeros((len(EMG_di_occ_peaks),))
for idx in range(len(P_occ_peaks)):
    start_i = P_occ_starts[idx]
    end_i = P_occ_ends[idx]
    start_plot_i = max([int(t_plot_start[idx] * vent_fs), 0])
    end_plot_i = min(
        [int(t_plot_end[idx] * vent_fs), len(x)-1])

    axis[0, idx].grid(True)
    axis[0, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                      x[start_plot_i:end_plot_i+1])
    axis[0, idx].plot(x_emg[start_plot_i:end_plot_i+1],
                      P_rolling_base_line[start_plot_i:end_plot_i+1])
    

    baseline_start_i = max([0, P_occ_peaks[idx] - 5*vent_fs])
    baseline_end_i = min([len(x) - 1, P_occ_peaks[idx] + 5*vent_fs])
    p_vent_max[idx] = max(P_rolling_base_line[baseline_start_i:baseline_end_i])

    popt, pcov = curve_fit(func, 
                        x_emg[start_i:end_i+1], 
                        -(x[start_i:end_i+1]-p_vent_max[idx]),
                        bounds=([0., x_emg[P_occ_peaks[idx]]-0.5, 0.], 
                                [100., x_emg[P_occ_peaks[idx]]+0.5, 0.5])
                        )
    
    PTP_occ_bell_error[idx] = np.trapz(
        np.sqrt((x[start_i:end_i+1] - 
        (-func(x_emg[start_i:end_i+1], *popt)+p_vent_max[idx])) **2),
        dx=1/vent_fs
    )
    axis[0, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                    -func(x_emg[start_plot_i:end_plot_i+1], *popt)
                    + P_rolling_base_line[start_plot_i:end_plot_i+1], 'g')

    axis[0, idx].plot(x_emg[P_occ_peaks[idx]], x[P_occ_peaks[idx]], '*r')
    axis[0, idx].plot(x_emg[start_i], x[start_i], '*r')
    axis[0, idx].plot(x_emg[end_i], x[end_i], '*r')


axis[0, 0].set_ylabel('Pocc (cmH2O)')

ETP_di_bell_error = np.zeros((len(EMG_di_occ_peaks),))
for idx in range(len(EMG_di_occ_peaks)):
    start_i = EMG_di_occ_starts_new[idx]
    end_i = EMG_di_occ_ends_new[idx]
    start_plot_i = max([int(t_plot_start[idx] * emg_fs), 0])
    end_plot_i = min([int(t_plot_end[idx] * emg_fs), len(y_di)-1])
    
    axis[1, idx].grid(True)
    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i], 
                      ecg_rms[start_plot_i+205:end_plot_i+205]/
                      max(ecg_rms[start_plot_i+205:end_plot_i+205])*
                      max(y_di[start_plot_i:end_i]), 
                      'tab:gray', linewidth=0.5)
    
    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                      y_di[start_plot_i:end_plot_i+1])
    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i+1],
                      emg_di_base_factor*
                      emg_di_rolling_base_line_aug[start_plot_i:end_plot_i+1])
    
    baseline_start_i = max([0, EMG_di_occ_peaks[idx] - 5*emg_fs])
    baseline_end_i = min(
        [len(y_di) - 1, EMG_di_occ_peaks[idx] + 5*emg_fs])
    y_di_min[idx] = min(y_di[baseline_start_i:baseline_end_i])

    # Too little samples to fit parameters to --> Add additional datapoints
    if end_i - start_i < 3:
        plus_idx = 3 - (end_i - start_i)
    else:
        plus_idx = 0
    popt, pcov = curve_fit(func, 
                    x_emg[start_i:end_i+1], 
                    y_di[start_i:end_i+1]-y_di_min[idx],
                    bounds=([0., x_emg[EMG_di_occ_peaks[idx]]-0.5, 0.], 
                            [60., x_emg[EMG_di_occ_peaks[idx]]+0.5, 0.5])
                    )

    ETP_di_bell_error[idx] = np.trapz(
        np.sqrt((y_di[start_i:end_i+1] - 
        (func(x_emg[start_i:end_i+1], *popt)+y_di_min[idx])) **2),
        dx=1/emg_fs
    )

    axis[1, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                      func(x_emg[start_plot_i:end_plot_i+1], *popt)
                      + y_di_min[idx], 'g')
    


    axis[1, idx].plot(x_emg[EMG_di_occ_peaks[idx]],
                      y_di[EMG_di_occ_peaks[idx]], '*r')
    axis[1, idx].plot(x_emg[start_i], y_di[start_i], '*r')
    axis[1, idx].plot(x_emg[end_i], y_di[end_i], '*r')

    axis[1, idx].set_ylim([0, 1.2*max(y_di[EMG_di_occ_peaks])])

    axis[1, idx].set_xlabel('t (s)')

axis[1, 0].set_ylabel('sEMGdi (uV)')

ETP_para_bell_error = np.zeros((len(EMG_para_occ_peaks),))
for idx in range(len(EMG_para_occ_peaks)):
    start_i = EMG_para_occ_starts_new[idx]
    end_i = EMG_para_occ_ends_new[idx]
    start_plot_i = max([int(t_plot_start[idx] * emg_fs), 0])
    end_plot_i = min([int(t_plot_end[idx] * emg_fs), len(y_para)-1])

    axis[2, idx].grid(True)
    axis[2, idx].plot(x_emg[start_plot_i:end_plot_i], 
                    ecg_rms[start_plot_i+205:end_plot_i+205]/
                    max(ecg_rms[start_plot_i+205:end_plot_i+205])*
                    max(y_para[start_plot_i:end_plot_i]), 
                    'tab:gray', linewidth=0.5)
    axis[2, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                      y_para[start_plot_i:end_plot_i+1])
    axis[2, idx].plot(x_emg[start_plot_i:end_plot_i+1],
                      emg_para_base_factor*
                      emg_para_rolling_base_line_aug[start_plot_i:end_plot_i+1])
    baseline_start_i = max([0, EMG_para_occ_peaks[idx] - 5*emg_fs])
    baseline_end_i = min(
        [len(y_para) - 1, EMG_para_occ_peaks[idx] + 5*emg_fs])
    y_para_min[idx] = min(y_para[baseline_start_i:baseline_end_i])

    # Too little samples to fit parameters to --> Add additional datapoints 
    if end_i - start_i < 3:
        plus_idx = 3 - (end_i - start_i)
    else:
        plus_idx = 0

    popt, pcov = curve_fit(func, 
                        x_emg[start_i:end_i+1+plus_idx], 
                        y_para[start_i:end_i+1+plus_idx]-y_para_min[idx],
                        bounds=([0., x_emg[EMG_para_occ_peaks[idx]]-0.5, 0.], 
                                [60., x_emg[EMG_para_occ_peaks[idx]]+0.5, 0.5])
                        )
    
    ETP_para_bell_error[idx] = np.trapz(
        np.sqrt((y_para[start_i:end_i+1] - 
        (func(x_emg[start_i:end_i+1], *popt)+y_para_min[idx])) **2),
        dx=1/emg_fs
    )

    axis[2, idx].plot(x_emg[start_plot_i:end_plot_i+1], 
                      func(x_emg[start_plot_i:end_plot_i+1], *popt)
                      +y_para_min[idx], 'g')

    axis[2, idx].plot(x_emg[EMG_para_occ_peaks[idx]],
                      y_para[EMG_para_occ_peaks[idx]], '*r')
    axis[2, idx].plot(x_emg[start_i], y_para[start_i], '*r')
    axis[2, idx].plot(x_emg[end_i], y_para[end_i], '*r')

    

    axis[2, idx].set_xlabel('t (s)')

axis[2, 0].set_ylabel('sEMGpara (uV)')


# Store values for later analysis

In [ ]:
fig_1.savefig(main_output_dir + '/' + patient + '/' + measurements[date_idx] 
              + '/' + measurements[date_idx] + '_' + patient + '_' 
              + measurement + '_timeplots_'+str(int(gate_width*1000))+'.png', 
              dpi=300)
fig_2.savefig(main_output_dir + '/' + patient + '/' + measurements[date_idx]  
              + '/' + measurements[date_idx] + '_' + patient + '_'
              + measurement + '_snapshots_'+str(int(gate_width*1000))+'.png', 
              dpi=300)
fig_3.savefig(main_output_dir + '/' + patient + '/' + measurements[date_idx]  
              + '/' + measurements[date_idx] + '_' + patient + '_' 
              + measurement + '_bell_errors_'+str(int(gate_width*1000))+'.png', 
              dpi=300)

fig_4.savefig(main_output_dir + '/' + patient + '/' + measurements[date_idx]  
              + '/' + measurements[date_idx] + '_' + patient + '_' 
              + measurement + '_Pocc_slopes_'+str(int(gate_width*1000))+'.png', 
              dpi=300)

In [ ]:
try:
    df
except NameError:
    df_exists = False
else:
    df_exists = True

if (len(big_data_list) == 0 
    or (df_exists and not (measurement in df['measurement'].to_list()))):
    print('Data of measurement ' + measurement + ' not yet added to the ' 
        + 'dataframe. Adding the data of:')
    patient, measurements[date_idx], measurement

    
    for idx in range(len(PTP_occs)):
        data_now = [patient, measurement, PEEP_set, PTP_occs[idx],
                    ETP_di_occs[idx], NME_di[idx], ETP_para_occs[idx], 
                    SNR_di[idx], SNR_para[idx], 
                    PTP_occ_baseline[idx], ETP_di_baseline[idx], 
                    ETP_para_baseline[idx],
                    ETP_di_baseline_old[idx], ETP_para_baseline_old[idx],
                    PTP_occ_bell_error[idx], ETP_di_bell_error[idx], 
                    ETP_para_bell_error[idx], 
                    P_peak_occs[idx], PTP_occs_peak[idx], 
                    PTP_occ_peak_baseline[idx],
                    t_delta_resp_efforts, t_delta_resp_efforts_volume, 
                    t_delta_di_med, t_delta_para_med, t_delta_ecg_med,
                    P_occ_peaks[idx]/vent_fs, 
                    P_occ_starts[idx]/vent_fs, 
                    P_occ_ends[idx]/vent_fs,
                    EMG_di_occ_peaks[idx]/emg_fs, 
                    EMG_di_occ_starts_new[idx]/emg_fs, 
                    EMG_di_occ_ends_new[idx]/emg_fs,
                    EMG_para_occ_peaks[idx]/emg_fs, 
                    EMG_para_occ_starts_new[idx]/emg_fs, 
                    EMG_para_occ_ends_new[idx]/emg_fs,
                    dP_up_10[idx], dP_up_90[idx], double_dips[idx],
                    ]
        
        big_data_list.append(data_now)
else:
    print('Data of PEEP step ' + measurement + ' already added to the ' 
          + 'dataframe. Don''t add the data twice!')
    


In [ ]:
big_data_list

In [ ]:
columns_now = ['patient', 'measurement', 'PEEP_set', 'PTP_occs', 'ETP_di_occs', 
               'NME_di', 'ETP_para_occs', 'SNR_di', 'SNR_para',
               'PTP_occ_baseline', 'ETP_di_baseline', 'ETP_para_baseline', 
               'ETP_di_baseline_old', 'ETP_para_baseline_old', 
               'PTP_occ_bell_SE', 'ETP_di_bell_SE', 'ETP_para_bell_SE', 
               'P_peak_occs', 'PTP_occs_peak', 'PTP_occ_peak_baseline', 
               't_delta_resp_efforts', 't_delta_resp_efforts_volume', 
               't_delta_di_med', 't_delta_para_med', 't_delta_ecg_med',
               't_Pocc_peak', 't_Pocc_start', 't_Pocc_end',
               't_di_peak', 't_di_start', 't_di_end',
               't_para_peak', 't_para_start', 't_para_end',
               'dP_up_10', 'dP_up_90', 'double_dip'
               ]

df = pd.DataFrame(big_data_list, columns=columns_now)
df

In [ ]:
bp = df.plot.scatter('PEEP_set', 'NME_di')
bp.set_ylabel('NME (cmH2O/uV)')
bp.set_xlabel('PEEP (cmH2O)')
bp.set_title('')

In [ ]:
# Generate read-me file
if gate_twice == True:
    N_gated = 2
else:
    N_gated = 1

context = (patient + ' / ' + measurements[date_idx] + ' / ' + measurement + '\n'
        +'t_start: ' + str(start) + ' s' + '\n'
        +'t_end: ' + str(end) + ' s' + '\n'
        +'gate_width: ' + str(int(gate_width*1000)) + ' ms' + '\n'
        +'gate_threshold: ' + str(peak_fraction) + '' + '\n'
        +'gate_ECG_shift: ' + str(ECG_shift) + ' samples' + '\n'
        +'time_shift: ' + str(vent_delay) + ' s' + '\n'
        +'Pocc_prominence_factor: ' + str(Pocc_prominence_factor) 
        + '' + '\n'
        +'EMG_di_prominence_factor: ' + str(EMG_di_prominence_factor) 
        + '' + '\n'
        +'EMG_para_prominence_factor: ' + str(EMG_para_prominence_factor) 
        + '' + '\n'
        +'PEEP_set: ' + str(PEEP_set) + '' + '\n' \
        +'N_gated: ' + str(N_gated) + '' + '\n' \
        +'augmented_percentile: ' + str(augmented_perc) + ' %' + '\n'
        +'emg_baseline_factor:' + str(emg_baseline_factor) + '\n'
        +'emg_di_baseline_factor:' + str(emg_di_base_factor) + '\n'
        +'emg_para_baseline_factor:' + str(emg_para_base_factor) + '\n'
        +'analysis_date: ' + str(datetime.now()) + ''
        )

with open(main_output_dir + '/' + patient + '/' + measurements[date_idx] + '/' 
          + measurements[date_idx] + '_' + patient + '_' + measurement 
          + '_readme.txt', 'w') as f:
    f.write(context)

print(context)

In [ ]:
# Move on to the next measurement
if measurement in df['measurement'].to_list():
    if measurement_idx >= len(list_of_numbers_strung):
        print('Data of measurement ' + measurement + ' added to the dataframe. '
              + 'You can continue to the saving the overall data!')
    elif measurement != list_of_numbers_strung[measurement_idx]:
        print('Data of measurement ' + measurement + ' added to the dataframe. '
              + 'You can continue to the next PEEP step!')
    else:
        measurement_idx += 1
        if measurement_idx >= len(list_of_numbers_strung):
            print('Data of measurement ' + measurement + ' added to the '
                + 'dataframe. You can continue to the saving the overall data!')
        else:
            print('Data of measurement ' + measurement + ' added to the '
                + 'dataframe. You can continue to the next PEEP step!')
else:
    print('Data of measurement ' + measurement + ' not yet added to the ' 
        + 'dataframe!')

['To next PEEP step'](#section_peep_step)

In [ ]:
# Store session data if complete
if len(set(df['PEEP_set'].values)) < 4:
    print('Warning: Not 4 PEEP settings evaluated yet!')
else:
    
    df.to_csv(main_output_dir + '/' + patient + '/' + measurements[date_idx]  
              + '/' + measurements[date_idx] + '_' + patient
              +'_NME_output_gate_'+str(int(gate_width*1000)) + '.csv')

    bp.figure.savefig(main_output_dir + '/' + patient + '/' 
                      + measurements[date_idx]  + '/' + measurements[date_idx] 
                      + '_' + patient + '_gate_'+str(int(gate_width*1000)) 
                      + '_scatter.png', dpi=300)
    
    print('Notification: Data of 4 PEEP settings stored!')
    print(measurements[date_idx])

In [ ]:
# Move on to the next measurement date if data saved

if not os.path.exists(main_output_dir + '/' + patient + '/' 
                      + measurements[date_idx]  + '/' + measurements[date_idx] 
                      + '_' + patient + '_NME_output_gate_'
                      + str(int(gate_width*1000))+'.csv'):
    print('Warning: Data not stored! Do not progress to next date!')
elif len(set(df['PEEP_set'].values)) < 4:
    print('Warning: Data not stored! Do not progress to next date!')
else:
    date_idx += 1
    if date_idx < len(measurements):
        print('Notification: Data appropriately stored. '
              +'You may progress to next date!')
    else:
        if patient == patients[patient_idx]:
            patient_idx += 1
            date_idx = 0
        print('Notification: Data appropriately stored. \n'+
              'This was the last measurement of this patient. '
              +'You may progress to next patient!')

In [ ]:
# # Save data anyway:
# df.to_csv(main_output_dir + '/' + patient + '/' + measurements[date_idx]   + '/' 
#           + measurements[date_idx]  + '_' + patient + '_NME_output_gate_'
#           +str(int(gate_width*1000))+'.csv')

# bp.figure.savefig(main_output_dir + '/' + patient + '/' + measurements[date_idx]   
#                   + '/' + measurements[date_idx]  + '_' + patient + '_gate_' 
#                   + str(int(gate_width*1000))+'_scatter.png', dpi=300)

In [ ]:
# # Move on to the next measurement date
# date_idx += 1

In [ ]:
# # Move on to the previous measurement date
# date_idx -= 1

In [ ]:
# date_idx

['To next measurement date'](#section_date_step)

In [ ]:
# Move on to the next patient
# patient_idx += 1

In [ ]:
# # Move on to the previous patient
# patient_idx -= 1

['To next patient'](#section_patient_step)

In [ ]:
# gate_twice = True